In [1]:
import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

Entrez.email = "l.singh@intbio.org"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

39737


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [39]:
# add_histone = (
#     "INSERT INTO histone "
#     "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
#     "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
# )
# add_histone_description = (
#     "INSERT INTO histone_description "
#     "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
#     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
# )
# add_publication = (
#     "INSERT INTO publication "
#     "(id, title, doi, author, year) "
#     "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
# add_alternate_names = (
#     "INSERT INTO alternative_name "
#     "(name, taxonomy, gene, splice, histone) "
#     "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
# )
# add_histone_has_publication = (
#     "INSERT INTO histone_has_publication "
#     "(histone_id, publication_id) "
#     "VALUES (%s, %s)"
# )

# Update description for macroH2A.1_(Mammalia)

In [10]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='macroH2A.1_(Mammalia)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,expression,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats
0,macroH2A.1_(Mammalia),variant,Mammalia,40674,81,macroH2A,81,macroH2A.1_(Mammalia) is a protein encoded by ...,null,null,...,null,null,null,null,null,null,null,null,null,null


In [11]:
summary = "macroH2A.1_(Mammalia) is believed to be a protein that is specific to mammals, as it is a protein that is encoded by one of the macroH2A genes found in mammals (which corresponds to the macroH2A.1 gene in humans). This gene further produces two splice variants macroH2A.1.s1 and macroH2A.1.s2 by inclusion of mutually exclusive exons encoding a region within the macro domain. The macro domain of macroH2A.1.s1 isoform is capable of binding NAD+ derived metabolites, such as ADP-ribose and poly-ADP-ribose, while other isoforms (including those produced by macroH2A.2 gene homologs) do not have this ability [sun_histone_2019]."
query = (
    f"UPDATE histone_description SET summary='{summary}' "
    "WHERE id=81"
)
print(query)

UPDATE histone_description SET summary='macroH2A.1_(Mammalia) is believed to be a protein that is specific to mammals, as it is a protein that is encoded by one of the macroH2A genes found in mammals (which corresponds to the macroH2A.1 gene in humans). This gene further produces two splice variants macroH2A.1.s1 and macroH2A.1.s2 by inclusion of mutually exclusive exons encoding a region within the macro domain. The macro domain of macroH2A.1.s1 isoform is capable of binding NAD+ derived metabolites, such as ADP-ribose and poly-ADP-ribose, while other isoforms (including those produced by macroH2A.2 gene homologs) do not have this ability [sun_histone_2019].' WHERE id=81


In [12]:
cursor.execute(query)

In [13]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='macroH2A.1_(Mammalia)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,expression,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats
0,macroH2A.1_(Mammalia),variant,Mammalia,40674,81,macroH2A,81,macroH2A.1_(Mammalia) is believed to be a prot...,null,null,...,null,null,null,null,null,null,null,null,null,null


In [14]:
# Make sure data is committed to the database
conn.commit()

# Update description for macroH2A.2_(Mammalia)

In [6]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='macroH2A.2_(Mammalia)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,expression,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats
0,macroH2A.2_(Mammalia),variant,Mammalia,40674,82,macroH2A,82,macroH2A.2_(Mammalia) is a protein encoded by ...,null,null,...,null,null,null,null,null,null,null,null,null,null


In [8]:
summary = "macroH2A.2_(Mammalia) is believed to be a protein that is specific to mammals, as it is a protein that is encoded by one of the macroH2A genes found in mammals (which corresponds to macroH2A.2 gene in humans). The macro domain of macroH2A.2 variant is not capable of binding NAD+ derived metabolites [sun_histone_2019]."
query = (
    f"UPDATE histone_description SET summary='{summary}' "
    "WHERE id=82"
)
print(query)

UPDATE histone_description SET summary='macroH2A.2_(Mammalia) is believed to be a protein that is specific to mammals, as it is a protein that is encoded by one of the macroH2A genes found in mammals (which corresponds to macroH2A.2 gene in humans). The macro domain of macroH2A.2 variant is not capable of binding NAD+ derived metabolites [sun_histone_2019].' WHERE id=82


In [9]:
cursor.execute(query)

In [10]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='macroH2A.2_(Mammalia)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,expression,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats
0,macroH2A.2_(Mammalia),variant,Mammalia,40674,82,macroH2A,82,macroH2A.2_(Mammalia) is believed to be a prot...,null,null,...,null,null,null,null,null,null,null,null,null,null


In [11]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [7]:
cursor.close()
conn.close()
tunnel.stop()